In [1]:
YEAR, EVALUATION = 2019, 2

In [2]:
from IPython.core.display import display, HTML
display(HTML(f'<h1>ANÁLISIS DE LA {EVALUATION}ª EVALUACIÓN</h1>'))
display(HTML(f'<h2>IES Puerto de la Cruz - Telesforo Bravo | Curso {YEAR}-{YEAR + 1}</h2>'))

- [CARGA DE DATOS](#CARGA-DE-DATOS)
- [RATIOS](#RATIOS)
- [ÉXITO ESCOLAR](#%C3%89XITO-ESCOLAR)
- [COMPETENCIAS BÁSICAS](#COMPETENCIAS-B%C3%81SICAS)
- [ABSENTISMO](#ABSENTISMO)
- [GESTIÓN DE LA CONVIVENCIA](#GESTI%C3%93N-DE-LA-CONVIVENCIA)
- [RELACIÓN ENTRE VARIABLES](#RELACI%C3%93N-ENTRE-VARIABLES)
- [GRUPOS MÁGICOS](#GRUPOS-M%C3%81GICOS)

In [3]:
import os
import sys
import re

import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
from matplotlib import pyplot as plt
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import colorlover as cl
from IPython.core.display import display, HTML

sys.path.append('..')
from services import loaders, charts, myplotly, utils

init_notebook_mode(connected=True)

In [4]:
# Estilo propio para dataframes
with open('../custom.css') as f:
    css_rules = ''.join(f.readlines())
HTML('<style>' + css_rules + '</style>')

In [5]:
%%javascript
// Deshabilitar el auto-scrolling en todo el notebook
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

# CARGA DE DATOS

In [6]:
df, df_bc, labels = loaders.load_data(YEAR, EVALUATION)

In [7]:
df.head()

etapa  éxito  absentismo_justificado  \
curso evaluación grupo                                        
C1718 E1         ESO1A   ESO   32.0                    3.49   
                 ESO1B   ESO   43.5                    2.41   
                 ESO1C   ESO   44.0                    0.72   
                 ESO1D   ESO   46.2                    0.54   
                 ESO2A   ESO   24.1                    1.54   

                        absentismo_injustificado  partes  \
curso evaluación grupo                                     
C1718 E1         ESO1A                      5.48     2.0   
                 ESO1B                      3.77     0.0   
                 ESO1C                      4.22     3.0   
                 ESO1D                      2.58     3.0   
                 ESO2A                      4.52    10.0   

                        suspensión_asistencia  ratio  absentismo  éxito_abs  \
curso evaluación grupo                                                        
C1718 E1         ESO1A                    0.0     25        8.97        8.0   
                 ESO1B                    0.0     22        6.18       10.0   
                 ESO1C                    0.0     25        4.94       11.0   
                 ESO1D                    0.0     27        3.12       12.0   
                 ESO2A                    3.0     29        6.06        7.0   

                            ccbb  
curso evaluación grupo            
C1718 E1         ESO1A  4.685714  
                 ESO1B  4.746429  
                 ESO1C  5.242857  
                 ESO1D  4.775000  
                 ESO2A  4.614286

In [8]:
df_bc.head()

nivel  item  PA  AD  MA  EX  marca
curso evaluación grupo                                   
C1718 E1         ESO1A  1ESO    CL  36  56   8   0    4.3
                 ESO1A  1ESO  CMCT  24  56  20   0    4.9
                 ESO1A  1ESO    CD   8  80  12   0    5.1
                 ESO1A  1ESO   AAP  28  64   8   0    4.5
                 ESO1A  1ESO   CSC  16  76   8   0    4.8

In [9]:
# recurrent variables
stages = df['etapa'].unique()
years = df.index.levels[0]
current_year, current_eval = labels[-1]

In [10]:
# dataframe for current year and evaluation
dfc = df.loc[labels[-1]]
dfc_bc = df_bc.loc[labels[-1]]

In [11]:
# dataframe for comparisons with last evaluation
df_comp = dfc.merge(df.loc[labels[-2]], left_index=True, right_index=True)

df_bc_last_eval_new_index = df_bc.loc[labels[-2]].reset_index().set_index(['grupo', 'item'])
dfc_bc_new_index = dfc_bc.reset_index().set_index(['grupo', 'item'])
df_bc_comp = dfc_bc_new_index.merge(df_bc_last_eval_new_index, left_index=True, right_index=True)
df_bc_comp = df_bc_comp.reset_index().set_index(['grupo'])

# RATIOS

In [12]:
# filter for current evaluation
df_ce = df.xs(current_eval, level='evaluación')

## Evolución del número total de alumnado por niveles

In [13]:
myplotly.num_students_evolution(df_ce, stages, years)

## Evolución de la ratio de alumnado por niveles

In [14]:
myplotly.num_students_evolution(df_ce, stages, years, ratio=True)

# ÉXITO ESCOLAR

<div class="alert alert-success" role="alert">
    ÉXITO ESCOLAR = <b>0 suspensos</b>
</div>

## ÉXITO EN VALORES ABSOLUTOS

In [15]:
myplotly.hbar((dfc['ratio'], dfc['éxito_abs']), dfc.index, trace_names=('Ratio', 'Éxito'))

## ESO

In [16]:
dfc_ESO = loaders.get_data_by_stages(dfc, 'ESO')
myplotly.bar_simple(dfc_ESO.index, dfc_ESO['éxito'])

In [17]:
dfc_ESO.mean()

éxito                       38.135714
absentismo_justificado       2.249286
absentismo_injustificado     4.607143
partes                       0.000000
suspensión_asistencia        0.000000
ratio                       22.928571
absentismo                   6.856429
éxito_abs                    9.000000
ccbb                         4.801786
dtype: float64

## FPB

In [18]:
dfc_FPB = loaders.get_data_by_stages(dfc, 'FPB')
myplotly.bar_simple(dfc_FPB.index, dfc_FPB['éxito'])

In [19]:
dfc_FPB.mean()

éxito                       30.500
absentismo_justificado       7.100
absentismo_injustificado    20.715
partes                       0.000
suspensión_asistencia        0.000
ratio                       11.000
absentismo                  27.815
éxito_abs                    4.000
ccbb                           NaN
dtype: float64

## BACHILLERATO

In [20]:
dfc_BACH = loaders.get_data_by_stages(dfc, 'BACH')
myplotly.bar_simple(dfc_BACH.index, dfc_BACH['éxito'])

In [21]:
dfc_BACH.mean()

éxito                       34.0500
absentismo_justificado       2.5500
absentismo_injustificado     4.4125
partes                       0.0000
suspensión_asistencia        0.0000
ratio                       21.0000
absentismo                   6.9625
éxito_abs                    7.0000
ccbb                            NaN
dtype: float64

## CICLOS FORMATIVOS DE GRADO MEDIO

In [22]:
dfc_CFGM = loaders.get_data_by_stages(dfc, 'CFGM')
myplotly.bar_simple(dfc_CFGM.index, dfc_CFGM['éxito'])

In [23]:
dfc_CFGM.mean()

éxito                       62.066667
absentismo_justificado       2.666667
absentismo_injustificado    14.185000
partes                       0.000000
suspensión_asistencia        0.000000
ratio                       13.666667
absentismo                  16.851667
éxito_abs                    7.833333
ccbb                              NaN
dtype: float64

## CICLOS FORMATIVOS DE GRADO SUPERIOR

In [24]:
dfc_CFGS = loaders.get_data_by_stages(dfc, 'CFGS')
myplotly.bar_simple(dfc_CFGS.index, dfc_CFGS['éxito'])

In [25]:
dfc_CFGS.mean()

éxito                       75.600000
absentismo_justificado       1.853636
absentismo_injustificado     7.980000
partes                       0.000000
suspensión_asistencia        0.000000
ratio                       19.454545
absentismo                   9.833636
éxito_abs                   14.181818
ccbb                              NaN
dtype: float64

## GLOBAL

In [26]:
dfc_summary = dfc.groupby('etapa').mean().loc[stages]
myplotly.bar_simple(dfc_summary.index, dfc_summary['éxito'])

## COMPARATIVA CON LOS ÚLTIMOS CURSOS

In [27]:
aux = df.groupby(['etapa', 'curso', 'evaluación']).mean().reset_index().set_index('etapa')
series = {year: aux.loc[stages].query(f'curso == "{year}" & evaluación == "{labels[-1][1]}"')['éxito'].values
          for year in years}

myplotly.cbar(stages, series)

In [28]:
mean_success = dfc['éxito'].mean()
display(HTML(f'''
<div class='alert alert-info' role="alert">
    Éxito global medio: <b>{mean_success:.2f}</b>%
</div>
'''))

In [29]:
if EVALUATION > 1:
    display(HTML('<h2>EVOLUCIÓN DE GRUPOS CON RESPECTO A LA EVALUACIÓN ANTERIOR</h2>'))

    aux = df_comp['éxito_x'] - df_comp['éxito_y']
    myplotly.dbar(aux.index, aux)

# COMPETENCIAS BÁSICAS

**AAP**: Aprender a aprender.  
**CD**: Competencia digital.  
**CEC**: Conciencia y expresiones culturales.  
**CL**: Comunicación lingüística.  
**CMCT**: Competencia matemática y competencias básicas en ciencia y tecnología.  
**CSC**: Competencias sociales y cívicas.  
**SIEE**: Sentido de iniciativa y espíritu emprendedor.  

#### Valor resumen por competencia básica:
\begin{equation}
MARCA = 0.025 * \mathbb{PA} + 0.050 * \mathbb{AD} + 0.075 * \mathbb{MA} + 0.1 * \mathbb{EX}
\end{equation}

, donde $\mathbb{PA}$, $\mathbb{AD}$, $\mathbb{MA}$ y $\mathbb{EX}$ son los porcentajes de adquisición de la competencia básica en cuestión.

## ESTUDIO POR NIVELES Y COMPETENCIA

In [31]:
myplotly.bc_bar(dfc_bc)

## Análisis simplificado por competencias básicas

In [32]:
dfc_bc_marked = dfc_bc.groupby(['nivel', 'item']).mean().reset_index().pivot('item', 'nivel', 'marca')

In [33]:
colors = cl.scales['4']['div']['RdYlGn']
q1 = np.quantile(dfc_bc_marked.values, q=0.1)
q2 = 5
q3 = np.quantile(dfc_bc_marked.values, q=0.9)
if q3 > q2:
    new_thresholds = [q1, q2, q3]
else:
    new_thresholds = [q1, q2]
thresholds = utils.normalize_thresholds(dfc_bc_marked, new_thresholds)
colorscale = utils.make_colorscale(colors, thresholds, discrete=True)

myplotly.heatmap(dfc_bc_marked.index, dfc_bc_marked.columns, dfc_bc_marked.values.T, colorscale)

### Nivel competencial resumen por nivel

In [34]:
aux = dfc_bc['marca'].groupby(dfc_bc['nivel']).mean()
myplotly.bar_simple(aux.index, aux, is_percentage=False, yaxis_range=[0, 10], mark_colors=True)

### Nivel competencial resumen por competencias básicas

In [35]:
aux = dfc_bc['marca'].groupby(dfc_bc['item']).mean()
myplotly.bar_simple(aux.index, aux, is_percentage=False, yaxis_range=[0, 10], mark_colors=True)

### Máximo nivel competencial

In [36]:
dfc_bc.iloc[dfc_bc.reset_index()['marca'].idxmax()][['item', 'marca']]

item      CSC
marca    5.85
Name: ESO1D, dtype: object

### Mínimo nivel competencial

In [37]:
dfc_bc.iloc[dfc_bc.reset_index()['marca'].idxmin()][['item', 'marca']]

item        CL
marca    3.325
Name: ESO2C, dtype: object

## Evolución de los niveles competenciales

In [38]:
# filter by evaluation
df_bc_same_eval = df_bc.xs(labels[-1][1], level='evaluación')
myplotly.bc_evolution(df_bc_same_eval)

In [46]:
if EVALUATION > 1:
    display(HTML('<h2>Evolución competencial con respecto a la evaluación anterior</h2>'))
    
    df_bc_comp['delta'] = df_bc_comp['marca_x'] - df_bc_comp['marca_y']
    df_bc_comp_marked = df_bc_comp.groupby(['nivel_x', 'item']).mean().reset_index().pivot('item', 'nivel_x', 'delta')

    min_diff_value = df_bc_comp_marked.values.min()
    max_diff_value = df_bc_comp_marked.values.max()
    print(f'El valor mínimo del diferencial es: {min_diff_value}')
    print(f'El valor máximo del diferencial es: {max_diff_value}')
    
    prompt = '''
Introduzca los 3 valores umbral (entre máximo y mínimo) separados por comas para la escala de colores.
El valor 0 debería ocupar la posición central:'''
    values = input(prompt)
    
    thresholds = [float(v) for v in re.split(' *, *', values)]

    colors = cl.scales['4']['div']['RdYlGn']
    thresholds = utils.normalize_thresholds(df_bc_comp_marked, thresholds)
    colorscale = utils.make_colorscale(colors, thresholds=thresholds, discrete=True)

    myplotly.heatmap(df_bc_comp_marked.index, df_bc_comp_marked.columns, df_bc_comp_marked.values.T, colorscale)

El valor mínimo del diferencial es: -0.4249999999999998
El valor máximo del diferencial es: 1.4500000000000002

Introduzca los 3 valores umbral (entre máximo y mínimo) separados por comas para la escala de colores.
El valor 0 debería ocupar la posición central:-0.2, 0, 1


## Correlación entre competencias básicas

> Se considera que dos variables cuantitativas están correlacionadas cuando los valores de una de ellas varían sistemáticamente con respecto a los valores homónimos de la otra: si tenemos dos variables (A y B) existe correlación entre ellas si al disminuir los valores de A lo hacen también los de B y viceversa. La correlación entre dos variables no implica, por sí misma, ninguna relación de causalidad.

Fuente: [Wikipedia](https://es.wikipedia.org/wiki/Correlaci%C3%B3n)

In [47]:
aux = dfc_bc.reset_index().pivot(index='grupo', columns='item', values='marca').corr()
aux.values[np.triu_indices(aux.shape[0])] = np.NaN

colorscale = utils.make_colorscale(cl.scales['8']['div']['PiYG'])
myplotly.heatmap(aux.index, aux.columns, aux.values.T, colorscale=colorscale)

## Mejores y peores valoraciones de competencias básicas por grupo

In [48]:
myplotly.bc_extremes(dfc_bc)

# ABSENTISMO

<div class="alert alert-success" role="alert">
    ABSENTISMO = <b>Faltas justificadas + Faltas injustificadas</b>
</div>

**Se va a estudiar el ABSENTISMO ACUMULADO desde el 1 de septiembre hasta la fecha de finalización del trimestre de análisis.**

## ABSENTISMO TOTAL vs ABSENTISMO JUSTIFICADO

In [49]:
myplotly.hbar((dfc['absentismo'], dfc['absentismo_justificado']), dfc.index,
              trace_names=('Abs. Total (%)', 'Abs. Justificado (%)'))

## ESO

In [50]:
myplotly.bar_simple(dfc_ESO.index, dfc_ESO['absentismo'], colormap='Reds')

# FPB

In [51]:
myplotly.bar_simple(dfc_FPB.index, dfc_FPB['absentismo'], colormap='Reds')

## BACHILLERATO

In [52]:
myplotly.bar_simple(dfc_BACH.index, dfc_BACH['absentismo'], colormap='Reds')

## CICLOS FORMATIVOS DE GRADO MEDIO

In [53]:
myplotly.bar_simple(dfc_CFGM.index, dfc_CFGM['absentismo'], colormap='Reds')

## CICLOS FORMATIVOS DE GRADO SUPERIOR

In [54]:
myplotly.bar_simple(dfc_CFGS.index, dfc_CFGS['absentismo'], colormap='Reds')

## GLOBAL

In [55]:
myplotly.bar_simple(dfc_summary.index, dfc_summary['absentismo'], colormap='Reds')

## COMPARATIVA CON LOS ÚLTIMOS CURSOS

In [56]:
aux = df.groupby(['etapa', 'curso', 'evaluación']).mean().reset_index().set_index('etapa')
series = {year: aux.loc[stages].query(f'curso == "{year}" & evaluación == "{labels[-1][1]}"')['absentismo'].values
          for year in years}

myplotly.cbar(stages, series)

In [57]:
mean_absence = dfc['absentismo'].mean()
display(HTML(f'''
<div class='alert alert-info' role="alert">
    Absentismo global medio: <b>{mean_absence:.2f}</b>%
</div>
'''))

In [58]:
if EVALUATION > 1:
    display(HTML('<h2>EVOLUCIÓN DE GRUPOS CON RESPECTO A LA EVALUACIÓN ANTERIOR'))

    aux = df_comp['absentismo_x'] - df_comp['absentismo_y']
    myplotly.dbar(aux.index, aux, inverted_colors=True)

# GESTIÓN DE LA CONVIVENCIA

## PARTES DE GESTIÓN

In [59]:
dfc_reports = dfc.query('partes > 0')['partes']
myplotly.bar_simple(dfc_reports.index, dfc_reports, colormap='Reds', is_percentage=False)

## COMPARATIVA CON LOS ÚLTIMOS CURSOS

In [60]:
aux = df.groupby(['etapa', 'curso', 'evaluación']).sum().reset_index().set_index('etapa')
series = {year: aux.loc[stages].query(f'curso == "{year}" & evaluación == "{labels[-1][1]}"')['partes'].values
          for year in years}

myplotly.cbar(stages, series, is_percentage=False)

In [61]:
total_reports = dfc_reports.sum()
display(HTML(f'''
<div class='alert alert-info' role="alert">
    Total de partes de gestión: <b>{total_reports:.0f}</b>
</div>
'''))

In [62]:
if EVALUATION > 1:
    display(HTML('<h2>EVOLUCIÓN DE GRUPOS CON RESPECTO A LA EVALUACIÓN ANTERIOR'))

    aux = df_comp['partes_x'] - df_comp['partes_y']
    myplotly.dbar(aux.index, aux, is_percentage=False, inverted_colors=True)

## SUSPENSIÓN DEL DERECHO DE ASISTENCIA

In [63]:
dfc_non_attendance = dfc.query('suspensión_asistencia > 0')['suspensión_asistencia']
myplotly.bar_simple(dfc_non_attendance.index, dfc_non_attendance, colormap='Reds', is_percentage=False)

## COMPARATIVA CON LOS ÚLTIMOS CURSOS

In [64]:
aux = df.groupby(['etapa', 'curso', 'evaluación']).sum().reset_index().set_index('etapa')
series = {year: aux.loc[stages].query(f'curso == "{year}" & evaluación == "{labels[-1][1]}"')['suspensión_asistencia'].values
          for year in years}

myplotly.cbar(stages, series, is_percentage=False)

In [65]:
total_non_attendance = dfc_non_attendance.sum()
display(HTML(f'''
<div class='alert alert-info' role="alert">
    Total de partes de gestión con <b>suspensión</b> del derecho de asistencia: <b>{total_non_attendance:.0f}</b>
</div>
'''))

# RELACIÓN ENTRE VARIABLES

## RELACIÓN `éxito-competencias básicas`

> **NOTA**: Análisis únicamente con grupos de la ESO.

In [66]:
myplotly.scatter(dfc_ESO['éxito'], dfc_ESO['ccbb'], dfc_ESO.index,
        x_title='Éxito', y_title='Competencias básicas')

In [67]:
corr = dfc_ESO.corrwith(dfc['éxito'])['ccbb']
display(HTML(f'''
    <div class='alert alert-success' role='alert'>
        COEFICIENTE DE CORRELACIÓN: <b>{corr:.4f}</b>
    </div>
'''))

## Detección de *outliers*

### Bajo éxito y alta adquisición de competencias básicas

In [68]:
dfc_ESO[(dfc_ESO['éxito'] < 20) & (dfc_ESO['ccbb'] > 5)][['éxito', 'ccbb']]

,éxito,ccbb
grupo,,


### Alto éxito y baja adquisición de competencias básicas

In [69]:
dfc_ESO[(dfc_ESO['éxito'] > 50) & (dfc_ESO['ccbb'] < 4)][['éxito', 'ccbb']]

,éxito,ccbb
grupo,,


## RELACIÓN `ratio-éxito`

In [70]:
myplotly.scatter(dfc['ratio'], dfc['éxito'], dfc.index,
        x_title='Ratio', y_title='Éxito')

In [71]:
corr = dfc.corrwith(dfc['ratio'])['éxito']
display(HTML(f'''
    <div class='alert alert-success' role='alert'>
        COEFICIENTE DE CORRELACIÓN: <b>{corr:.4f}</b>
    </div>
'''))

## Detección de *outliers*

### Baja ratio y bajo éxito

In [72]:
dfc[(dfc['ratio'] < 20) & (dfc['éxito'] < 20)][['ratio', 'éxito']]

,ratio,éxito
grupo,,
ESO2C,6,16.7
2FPB,7,14.3


### Alta ratio y alto éxito

In [73]:
dfc[(dfc['ratio'] > 25) & (dfc['éxito'] > 50)][['ratio', 'éxito']]

,ratio,éxito
grupo,,
ESO3A,26,53.8
1ALO,27,74.1
1GIT,28,67.9
1ASR,26,57.7


## RELACIÓN `absentismo-éxito`

In [74]:
myplotly.scatter(dfc['absentismo'], dfc['éxito'], dfc.index,
        x_title='Absentismo', y_title='Éxito')

In [75]:
corr = dfc.corrwith(dfc['absentismo'])['éxito']
display(HTML(f'''
    <div class='alert alert-success' role='alert'>
        COEFICIENTE DE CORRELACIÓN: <b>{corr:.4f}</b>
    </div>
'''))

## Detección de *outliers*

### Bajo absentismo y bajo éxito

In [76]:
dfc[(dfc['absentismo'] < 10) & (dfc['éxito'] < 15)][['absentismo', 'éxito']]

,absentismo,éxito
grupo,,


### Alto absentismo y alto éxito

In [77]:
dfc[(dfc['absentismo'] > 25) & (dfc['éxito'] > 50)][['absentismo', 'éxito']]

,absentismo,éxito
grupo,,


## RELACIÓN `partes-éxito`

In [78]:
myplotly.scatter(dfc['partes'], dfc['éxito'], dfc.index,
        x_title='Partes de gestión', y_title='Éxito')

/Users/sdelquin/.virtualenvs/analitica/lib/python3.8/site-packages/scipy/stats/_stats_mstats_common.py:130: RuntimeWarning:

invalid value encountered in double_scalars

/Users/sdelquin/.virtualenvs/analitica/lib/python3.8/site-packages/scipy/stats/_stats_mstats_common.py:142: RuntimeWarning:

divide by zero encountered in double_scalars



In [79]:
corr = dfc.corrwith(dfc['partes'])['éxito']
display(HTML(f'''
    <div class='alert alert-success' role='alert'>
        COEFICIENTE DE CORRELACIÓN: <b>{corr:.4f}</b>
    </div>
'''))

## Detección de *outliers*

### Bajo número de partes y bajo éxito

In [80]:
dfc[(dfc['partes'] < 10) & (dfc['éxito'] < 10)][['partes', 'éxito']]

,partes,éxito
grupo,,


### Alto número de partes y alto éxito

In [81]:
dfc[(dfc['partes'] > 15) & (dfc['éxito'] > 30)][['partes', 'éxito']]

,partes,éxito
grupo,,


# GRUPOS MÁGICOS

> Son aquellos que tienen mejores valores de éxito escolar, menor absentismo y menos partes de gestión.

\begin{equation}
magic = 0.6 · success + 0.2 · e^{-absence} + 0.2 · e^{-reports}
\end{equation}

In [82]:
df2 = dfc.copy()
df2['magic'] = 0.6 * df2['éxito'] + 0.2 * \
    np.exp(-df2['absentismo']) + 0.2 * np.exp(-df2['partes'])
magic = df2.sort_values('magic', ascending=False)

series = {
    'Éxito escolar': magic['éxito'].values,
    'Absentismo': magic['absentismo'].values,
    'Partes gestión': magic['partes'].values
}

myplotly.cbar(magic.index, series, is_percentage=False, barmode='stack')